In [1237]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import researchpy as rp
import scipy.stats as stats

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

In [1238]:
#Raw file
df_og = pd.read_excel("world_of_dc_survey.xlsx")
#Backup
df = df_og.copy()

#Map of all replaces and info about each question
big_data_dict = "dcumap.xlsx"
excel_file = pd.ExcelFile(big_data_dict)
sheet_names = excel_file.sheet_names

question_types = pd.read_excel(big_data_dict, sheet_name="Data Dictionary")
question_info_dict = dict(zip(
    question_types['Shortq'],
    zip(
        question_types['Type'], #Multi-select vs single select, write-in vs no write-in
        question_types['Other_added'], #For write-ins, were there enough different responses to warrant "Other"?
        question_types['Dichotomy'], #Check these (multi-select) responses! Some of them can't both be true
        question_types['Split'], #Broad y/n qs
        question_types['Changed'], #Response options changed
        question_types['Wrong'] #Question type changed to a select
        )))

bool_sheets = {}
multi_basic_sheets = {}
multi_write_sheets = {}
single_basic_sheets = {}
single_write_sheets = {}
write_in_sheets = {}
other_sheets = {}

all_sheet_types = [
    bool_sheets,
    multi_basic_sheets,
    multi_write_sheets,
    single_basic_sheets,
    single_write_sheets,
    write_in_sheets,
    other_sheets
]

wrong_cols = [] #These were write-ins for like a second before I changed my mind

changed_cols = [] #These had responses edited while the survey was live

split_me_cols = [] #Poorly constructed questions that were actually 2-in-1s

dichotomy_sheets = {}


threshold_keys = [ #Too many defaults that went unused
    "family_affil",
    "base",
    "race"
]

threshold_dict = {key: question_info_dict[key] for key in threshold_keys}

for col, (type, other_check, dichotomy, split, changed, wrong) in question_info_dict.items():
    if split == "T":
        split_me_cols.append(col)
    if changed == "T":
        changed_cols.append(col)
    if wrong == "T":
        wrong_cols.append(col)
    if dichotomy == "T":
        dichotomy_sheets[col] = type, other_check
    if type =="Bool":
        bool_sheets[col] = other_check, dichotomy
    elif type == "Multi-basic":
        multi_basic_sheets[col] = other_check, dichotomy
    elif type == "Multi-write":
        multi_write_sheets[col] = other_check, dichotomy
    elif type == "Single-basic":
        single_basic_sheets[col] = other_check, dichotomy
    elif type == "Single-write":
        single_write_sheets[col] = other_check, dichotomy
    elif type == "Write-in":
        write_in_sheets[col] = other_check, dichotomy
    elif type == "Other":
        other_sheets[col] = other_check, dichotomy

In [1239]:
# USEFUL GENERAL

def col_index_move(df, col, new_col):
    #Reorders the columns of the df, moving new_col to be after col

    df = df.copy() #Prevents a million slice errors
    col_index = df.columns.get_loc(col)
    columns = df.columns.tolist()

    new_index = col_index + 1
    current_index = df.columns.get_loc(new_col)
    if current_index == new_index:
        return df

    columns.remove(new_col)
    columns.insert(col_index + 1, new_col) 

    return df[columns]

def add_other_to_dict(dict):
    dict.update({"Other": 'other'})     #Adds other to dict

def zipit(sheet_name):
    sheet_dict =  pd.read_excel(big_data_dict, sheet_name, header=None)     #A quick little zip for dicts stored in 1st and 2nd col of sheet
    mydict = dict(zip(sheet_dict[0], sheet_dict[1]))

    return mydict

def filter_columns(df, keyword):
    return [col for col in df.columns if keyword in col]     #Finds just the cols that have keyword


# SINGLE WRITE IN

def mutate_to_other(df, col, valid_list, what_to_mutate_to): # Works single-select only!
    #Checks values in col. Are they in valid_list? They stay. Are they not? Mutates to what_to_mutate_to

    df.loc[:, col] = df[col].apply(lambda x: x if pd.isna(x) or x in valid_list else what_to_mutate_to)
    return df


# BLANK STATE

def goreset(df=df_og):
    #For messing with the df and you want to go back. This is the starting point of a usable df

    #add ID column
    df['ID'] = range(1, len(df) + 1)
    id_column = df.pop("ID")
    df.insert(0, "ID", id_column)

    #rename headers
    header_names = pd.read_excel(big_data_dict, sheet_name="Data Dictionary", header=None, skiprows=1)
    new_headers = header_names.iloc[:, 1].tolist()

    current_headers = df.columns.tolist()
    rename_dict = {current_headers[i+1]: new_headers[i] for i in range(len(new_headers))}
    df.rename(columns=rename_dict, inplace=True)

    #Fixing wrong write ins that changed to basic
    for col in wrong_cols:
        filtered_sheets = [sheet for sheet in sheet_names if sheet.startswith(col)]
        base = zipit(filtered_sheets[0])
        df = mutate_to_other(df, col, base, np.nan)

    #Converts cols to str for later stuff
    columns_to_convert = df.select_dtypes(include='object').columns
    for column in columns_to_convert:
        df[column] = df[column].astype(str)
        df[column].replace('nan', np.nan, inplace=True)

    return df


# MULTI SELECT NO WRITE INS!

def contains_other_pattern(row, patterns):
    # Checks row to see if it contains pattern, returning T/F or np.nan

    if isinstance(row, str):
        row_words = set([word.strip() for word in row.split(',')])
        pattern_words = set([pattern.strip() for pattern in patterns])
        return pattern_words.issubset(row_words)
    else:
        return np.nan

def multi_select_no_other(df, col, var_dict):
    #Creates a column for each item in var_dict, then goes row by row in col to check if the thing var_dict corresponds to apepars in the row or not

    var_dict = dict(reversed(var_dict.items())) #so the cute short friendly ones are the columns, and default responses are default responses
    for key, var in var_dict.items():
        patterns = key.split(',')
        var = f'{col}_{var}'
        
        df.loc[:, var] = df.loc[:, col].apply(lambda row: contains_other_pattern(row, patterns))

        df.loc[:, var] = df[var].replace({True: 1, False: 0})

        df = col_index_move(df, col, var)
        print(f'{var}: {patterns} done')
    return df


# PURE WRITE IN

def concat_write_in(row, col_list):
    #Returns a single concat of all the col_lists eg
    # col1 col2 col3
    # Red car desert
    # Red,car,desert

    non_null_columns = [col for col in col_list if not pd.isna(row[col])]
    
    if len(non_null_columns) == 0:
        return "Other"
    elif len(non_null_columns) == 1:
        return row[non_null_columns[0]]
    else:
        return ','.join(str(row[col]) for col in non_null_columns if not pd.isna(row[col]))


def pure_write_in(df, col, map, pauseforsingle=False):
    #Creates a concat col beside the col of what it maps to
    
    col_list = []
    n = 2
    while n < len(map.columns):
        col_list.append(map.columns[n])
        n += 1
    
    map[f'concat_{col}'] = map.apply(concat_write_in, col_list=col_list, axis=1)

    #For single write-in, want to avoid invalid responses slipping through
    #eg.
    # col1 concat_col1
    # obvioustroll Invalid
    
    # col1_combined
    # Invalid              # and not

    # col1 concat_col1
    # obvioustroll nan

    # col1_combined
    # obvioustroll

    if pauseforsingle == False:
        map[f'concat_{col}'] = map[f'concat_{col}'].replace('Invalid', np.nan) #For multis there will either be something there already or nan + nan = nan

    df = pd.merge(df, map[["ID", f'concat_{col}']], on='ID', how='left')

    df = col_index_move(df, col, f'concat_{col}')

    return df


# MULTI SELECT AND WRITE IN

def multiselect_get(df_2, col, base_dict, singleselect = False):
    #Create a new df to find the write ins for multiselect write ins!! Difficult bc base responses can have any number, plus may or may not have commas, silly regex
    #note: if soemone writes in exact defaults with nothing else it will eat the default. Should still show up for mapping and t/f so no biggie

    df_2 = df_2.copy()

    replace_dict = {re.escape(key): value for key, value in base_dict.items()} #Unfortunately some defaults have () and // and you'll live with your consequences

    #swaps w values to avoid cutting multi choice options like "No, I disagree" at comma
    df_2[f'{col}_replace'] = df_2[col].replace(replace_dict, regex=True)
    
    df_2 = df_2[["ID", f'{col}_replace']]
    df_2.dropna(subset=[f'{col}_replace'], inplace=True)

    def contains_values_not_in_base(cell_values, base):
        #Multi-select
        if pd.isna(cell_values):
            return False
        values = cell_values.split(',')
        for value in values:
            if value.strip() not in base:
                return True
        return False
    
    def values_not_in_base_SINGLE(cell_values, base):
        #Single-select
        if pd.isna(cell_values):
            return False
        if cell_values.strip() not in base:
            return True
        return False

    def remove_matches(text, exclude_list):
        #Cleans up things in exclude_list (base)
        for word in exclude_list:
            pattern = r"(^|,\s*|\n)" + re.escape(word) + r"(?=\s*(,|\n|$))"
            text = re.sub(pattern, lambda m: m.group(1) if m.group(1) else '', text)
        return text.strip()
    
    if singleselect == True:
        #Just get anything that isn't fully in the base_dict
        df_2 = df_2[df_2[f'{col}_replace'].apply(values_not_in_base_SINGLE, base = base_dict.values())]

    elif singleselect == False:
        #Keeps going until it hits something not in base
        df_2 = df_2[df_2[f'{col}_replace'].apply(contains_values_not_in_base, base = base_dict.values())]
        #Then gets rid of anything that IS in base so export is clean
        df_2[f'{col}_replace'] = df_2[f'{col}_replace'].apply(lambda x: remove_matches(x, base_dict.values()))
        df_2[f'{col}_replace'] = df_2[f'{col}_replace'].str.strip(",")

    return df_2


def singleselect_map(df, col, map):
    #Just put the single_write map beside its og col

    df = pure_write_in(df, col, map, True)

    df[f'{col}_combined'] = df[f'concat_{col}'].combine_first(df[col])
    df[f'{col}_combined'] = df[f'{col}_combined'].replace('Invalid', np.nan)
    df = col_index_move(df, col, f'{col}_combined')
    df.pop(f'concat_{col}')
    
    return df


def multiselect_map(df, col, map, base):
    #Need to: keep the base, but add in the new ones
    
    #Regex, for replacing
    base_noregex = {v : k for v, k in base.items()}
    base = {re.escape(key): value for key, value in base.items()}

    #Add the write ins
    df = pure_write_in(df, col, map)

    #Keep the base, include_list
    def remove_other_matches(text, include_list):
        if pd.isna(text):
            return np.nan
        values = text.split(',')
        filtered_values = [value for value in values if value.strip() in include_list]
        
        joint_filtered = ','.join(filtered_values)
        if joint_filtered.strip(): #kills blanks
            return joint_filtered
        else:
            return np.nan        
    
    #For catching base answers with commas eg "No, no one"
    df.loc[:, f'{col}'] = df[col].replace(base, regex=True)
    df.loc[:, f'{col}_filtered'] = df[col].apply(lambda text: remove_other_matches(text, base.values()))
    dict2 = {v : k for k, v in base_noregex.items()}

    df[f'{col}_filtered'] = df[f'{col}_filtered'].replace(dict2, regex=True)

    df = col_index_move(df, col, f'{col}_filtered')

    def combine_values(row):
        #Concat write ins and og, depending on which are empty etc

        filtered_value = row[f'{col}_filtered']
        concat_value = row[f'concat_{col}']
        
        if pd.isnull(filtered_value) and pd.isnull(concat_value):
            return np.nan
        
        parts = []
        if not pd.isnull(filtered_value):
            parts.append(str(filtered_value))
        if not pd.isnull(concat_value):
            parts.append(str(concat_value))
        
        return ', '.join(parts)

    df.loc[:, f'{col}_combined'] = df.apply(combine_values, axis=1)

    df = col_index_move(df, col, f'{col}_combined')
    df.pop(f'{col}_filtered')
    df.pop(f'concat_{col}')

    return df


def multiselect_tf(df, col, base, new_dict, add_other):
    #Call multi_select_no_other for the multiselect write_ins

    dict2 = {v : k for k, v in base.items()}
    dict2_back = {v : k for k, v in dict2.items()}

    var_dict = dict(dict2_back, **new_dict)
    if add_other == True:
        add_other_to_dict(var_dict)

    df = multi_select_no_other(df, f'{col}_combined', var_dict)

    return df


# MUTANTS

def always_only_never_resolve(df, col, always_list, only_list, never_list):
    #Fixes logical errors like, "I don't fight, I fight with my fists"
    #If anything in only_list, keeps only_list
    #If anything in always_list, keeps always_list

    def process_row(row):
        if isinstance(row, str):
            items = row.split(',')
            items = [item.strip() for item in items]
            if any(item in items for item in always_list):
                items = [item for item in items if item.strip() not in only_list and item.strip() not in never_list]
            elif any(item in items for item in only_list):
                items = [item for item in items if item.strip() not in never_list]
            return ', '.join(items)
        else:
            return np.nan

    df[col] = df[col].apply(process_row)

    return df


def replace_bool(cell):
    #Similar to always_only_never_resolve, fixes answers with both "Yes" and "No" for bools

    if "Yes" in str(cell): #Favour all YES options
        return "Yes"
    elif "No" in str(cell):
        return "No"
    return cell


def convert_to_other_threshold(df, col, multi=False):
    #Converts anything with few responses (defaults usually) to "Other" for easier analysis

    threshold = 0.025
    total_count = df[col].count()

    if multi == False:
        #Literally just count and check

        counts = df[col].value_counts()
        print(counts)
        bye_counts = {}
        bye_counts = {key: value for key, value in counts.items() if value < threshold * total_count}
        for key, value in bye_counts.items():
            escaped_key = re.escape(key)
            df[col] = df[col].replace(escaped_key, "Other", regex = True)
            print(f'{key} converted to other with {value} counts')

    elif multi == True:
        #Split, then count, then check, then replace

        split_cols = df[col].str.split(',', expand=True)
        for new_col in split_cols:
            split_cols[new_col] = split_cols[new_col].str.strip()
        stacked_cols = split_cols.stack()
        counts = dict(stacked_cols.value_counts())
        print(counts)
        bye_counts = {}
        bye_counts = {key: value for key, value in counts.items() if value < threshold * total_count}
        for key, value in bye_counts.items():
            escaped_key = re.escape(key)
            df[col] = df[col].replace(escaped_key, "Other", regex = True)
            print(f'{key} converted to other with {value} counts')

    return df


In [1240]:
df = goreset(df_og)
print(f'df has {len(df.columns)} cols and {len(df)} rows')
df.tail(7)

df has 20 cols and 46 rows


,ID,hero_or_villain,alignment_all,family_affil,team_affil,sidekick_status,base,fight_why,year_start,main_weapon,meta_powers,main_rival,lethal_thoughts,age,race,gender,orphan_status,trinity_1,trinity_2,trinity_3
39,40,Hero,I'm good,Flash,Nope,No,Manchester,I fight for justice,1940,Speedforce,Speedforce,"No, I don't","Never, maybe maybe one-offs",60.0,Human,M,No,NaN,NaN,NaN
40,41,vigilante,"Neutral, I'm good",Bat,Rogue Gallery,No,Gotham,I fight for fun :3,1940,Claws,No powers,"No, I don't","Only as last resort, don’t tell him",44.0,Human,F,No,Batman,NaN,Wonder Woman
41,42,Villain,I'm evil,Nope,Rogue Gallery,No,Gotham,"I fight for revenge, I fight for justice",1959,Tech,No powers,"No, I don't","Always, Only as last resort",40.0,Human,M,No,NaN,NaN,Batman
42,43,Villain,I'm evil,Nope,Rogue Gallery,No,Keystone (City),"I fight for fun :3, I fight for protection",1957,Tech,No powers,"Yes, very annoying","Never, Only if dangerous",45.0,Human,M,No,Superman,NaN,NaN
43,44,dating one?,Neutral,Nope,Nope,well I'm not a hero,Gotham,"Not applicable, I no longer/do not fight",2022,"Not applicable, I no longer/do not fight",No powers,"No, I don't",Only as last resort,20.0,Human,M,No,Batman,NaN,NaN
44,45,retired now!,"I'm good, Neutral",Nope,Young Justice,No,Happy Harbor,"Not applicable, I no longer/do not fight",2006,"Not applicable, I no longer/do not fight",Racial Abilities,"Yes, very dangerous",Only if dangerous,16.0,"Human, Ghost before",F,Yes,Batman,Superman,NaN
45,46,Hero,I'm good,Arrow,"Teen Titans, Young Justice",For a while there,Star (City),"Not applicable, I no longer/do not fight",2004,Arrows,No powers,"No, I don't",Only if dangerous,22.0,Human,F,No,Wonder Woman,NaN,NaN


In [1241]:
#Check for changed qs :')

bool_basic_group = bool_sheets.copy()
bool_basic_group.update(single_basic_sheets)

for col, (other_check, dichot) in bool_basic_group.items():
    print(df[col].value_counts())
    print("\n")

orphan_status
No     33
Yes    11
Name: count, dtype: int64


base
Gotham             11
Keystone (City)     5
New York            4
Star (City)         4
Metropolis          2
Manchester          2
San Francisco       2
Bludhaven           1
Capitol (City)      1
Portland            1
Los Angeles         1
Philadelphia        1
Brooklyn            1
Virginia            1
Shadowcrest         1
Baltimore           1
Coast (City)        1
El Paso             1
Watchtower          1
Smallville          1
Atlantis            1
Happy Harbor        1
Name: count, dtype: int64


year_start
1940    6
1959    4
2006    3
1964    2
1980    2
2014    2
1993    2
1941    2
2004    2
1968    2
1999    2
2016    1
1957    1
1965    1
1989    1
1956    1
1997    1
1943    1
1942    1
1992    1
1967    1
1977    1
1947    1
1955    1
1983    1
1996    1
1994    1
2022    1
Name: count, dtype: int64


main_rival
No, I don't            29
Yes, very dangerous    11
Yes, very annoying      6
Name: count, 

In [1242]:
#Handle changed qs that aren't multi

for col in changed_cols:
    
    filtered_sheets = [sheet for sheet in sheet_names if sheet.startswith(col)]
    base = zipit(filtered_sheets[0])
    dict2 = {v : k for k, v in base.items()}

    df.loc[:, f'{col}'] = df[col].replace(base)
    df.loc[:, f'{col}'] = df[col].replace(dict2, regex=True)

    print(df[col].value_counts())
    print("")

family_affil
Bat        9
Nope       9
Arrow      7
Flash      6
Super      4
Lantern    4
Wonder     2
Martian    2
Aqua       1
Beetle     1
Shazam     1
Name: count, dtype: int64

sidekick_status
No                     37
Yes                     4
No? ?                   1
tt Yes                  1
kind of, Yes, No        1
well I'm Not a hero     1
For a while there       1
Name: count, dtype: int64



In [1243]:
#Get the write-ins

#Run only once then leave turned off
#Then go into excel and tag each entry before running next bit

'''for col, (type, other_check, dichot, split, changed, wrong) in question_info_dict.items():
    if type=="Multi-write" or type=="Single-write":
        base = pd.read_excel(big_data_dict, sheet_name=col, header=None)
        base_dict = dict(zip(base[0], base[1]))

        if type == "Single-write":
            new_df = multiselect_get(df, col, base_dict, True)
        else:
            new_df = multiselect_get(df, col, base_dict)
        new_df.to_csv(f'{col}_replace.csv', index=False)
        print(f"{col} saved to csv!")'''


'for col, (type, other_check, dichot, split, changed, wrong) in question_info_dict.items():\n    if type=="Multi-write" or type=="Single-write":\n        base = pd.read_excel(big_data_dict, sheet_name=col, header=None)\n        base_dict = dict(zip(base[0], base[1]))\n\n        if type == "Single-write":\n            new_df = multiselect_get(df, col, base_dict, True)\n        else:\n            new_df = multiselect_get(df, col, base_dict)\n        new_df.to_csv(f\'{col}_replace.csv\', index=False)\n        print(f"{col} saved to csv!")'

In [1244]:
# Mapping

In [1245]:
for col, (other_check, dichotomy) in multi_write_sheets.items():

    filtered_sheets = [sheet for sheet in sheet_names if sheet.startswith(col)]
    base = zipit(filtered_sheets[0])

    map = pd.read_csv(f'Maps/{col}_replace.csv')

    df = multiselect_map(df, col, map, base)
    print(f"{col} mapped")

alignment_all mapped
lethal_thoughts mapped
race mapped


In [1246]:
changed_writes = ["sidekick_status"] #Something special for this changed_response

for col, (other_check, dichotomy) in single_write_sheets.items():

    map = pd.read_csv(f'Maps/{col}_replace.csv')

    df = singleselect_map(df, col, map)

    if col in changed_writes:
        #Dict whip back and forth to force everything down to 1 option

        filtered_sheets = [sheet for sheet in sheet_names if sheet.startswith(col)]
        base = zipit(filtered_sheets[0])

        base_noregex = {v : k for v, k in base.items()}
        base = {re.escape(key): value for key, value in base.items()}

        df.loc[:, f'{col}_combined'] = df[f'{col}_combined'].replace(base, regex=True)

        dict2 = {v : k for k, v in base_noregex.items()}
        df[f'{col}_combined'] = df[f'{col}_combined'].replace(dict2, regex=True)
    
    print(f"{col} mapped")

df.head()

hero_or_villain mapped
sidekick_status mapped
main_weapon mapped
gender mapped


,ID,hero_or_villain,hero_or_villain_combined,alignment_all,alignment_all_combined,family_affil,team_affil,sidekick_status,sidekick_status_combined,base,fight_why,year_start,main_weapon,main_weapon_combined,meta_powers,main_rival,lethal_thoughts,lethal_thoughts_combined,age,race,race_combined,gender,gender_combined,orphan_status,trinity_1,trinity_2,trinity_3
0,1,Hero,Hero,good,I'm good,Bat,"Teen Titans, Justice League",No,No,Bludhaven,I fight for justice,1940,Escrima,Other,No powers,"Yes, very dangerous","never, only_last, but you sometimes think it'd be easier","Never, Only as last resort, Other",29.0,human,Human,M,M,Yes,Batman,Superman,Wonder Woman
1,2,You tell me,NaN,"good, evil, Complicated, Lazarus","I'm good, I'm evil, Other",Bat,"Outlaws, League of Shadows",No,No,Gotham,"I fight for justice, I fight for revenge",1983,Gun,Gun,No powers,"Yes, very dangerous","always, only_danger, some people just deserve it","Always, Only if dangerous, Some exceptions",24.0,human,Human,M,M,Yes,Wonder Woman,Superman,Wonder Woman
2,3,Hero,Hero,good,I'm good,Bat,"Teen Titans, Young Justice",No? ?,No,Gotham,"I fight for justice, I fight for responsibility",1989,Bo staff,Other,No powers,"Yes, very dangerous",never,Never,20.0,human,Human,M,M,Yes,Batman,NaN,NaN
3,4,Hero,Hero,good,I'm good,Bat,"Teen Titans, League of Shadows",tt Yes,Yes,Gotham,"I fight for justice, I fight for redemption",2006,Knife,Knife,No powers,"No, I don't","only_last, only for people who really do deserve it","Only as last resort, Some exceptions",13.0,human,Human,M,M,No,Batman,Superman,NaN
4,5,Hero,Hero,good,I'm good,Bat,Nope,No,No,Gotham,"I fight for justice, I fight for responsibility",1992,Three-part staff,Other,No powers,"No, I don't","only_last, only_danger, but you can't take it back","Only as last resort, Only if dangerous, Other",21.0,human,Human,F,F,No,Wonder Woman,NaN,NaN


In [1247]:
# Post-mapping

In [1248]:
# Fix dichotomy

for col, (type, other) in dichotomy_sheets.items():
    always_list = []
    only_list = []
    never_list = []

    filtered_sheets = [sheet for sheet in sheet_names if sheet.startswith(col)]

    sheet_dict =  pd.read_excel(big_data_dict, col, header=None)
    mydict = dict(zip(sheet_dict[0],
                      zip(
                          sheet_dict[1],
                          sheet_dict[2])))
    
    if type == "Multi-write":
       col = f'{col}_combined'
       if len(filtered_sheets) ==2:
            sheet_dict_2 =  pd.read_excel(big_data_dict, filtered_sheets[1], header=None)
            mydict_2 = dict(zip(sheet_dict_2[0],
                    zip(
                        sheet_dict_2[1],
                        sheet_dict_2[2])))
            mydict.update(mydict_2)
       if other == "T": 
            mydict.update({"Other": ('other', 'only')}) #for repost-opinion. Other as "only if..." risks losing some, if always risks losing nevers

    for ans, (ans_short, dichot) in mydict.items():
        if dichot == "always":
            always_list.append(ans_short)
        elif dichot == "only":
            only_list.append(ans_short)
        elif dichot == "never":
            never_list.append(ans_short)

    print(always_list)
    print(only_list)
    print(never_list)

    replace_dict_regex = {re.escape(key): re.escape(value) for key, (value, dichot) in mydict.items()}
    
    df[col] = df[col].replace(replace_dict_regex, regex=True)

    df = always_only_never_resolve(df, col, always_list, only_list, never_list)

    back_dict = {ans_short:ans for ans, (ans_short, dichot) in mydict.items()}
    df[col] = df[col].replace(back_dict, regex=True)

    print(df[col].value_counts())
    #print(df[col].unique()) #Check for multi-writes w too many for value_counts
    print("\n")

[]
['batfam', 'batfam', 'wonderfam', 'aquafam', 'arrowfam', 'flashfam', 'superfam', 'martianfam', 'beetlefam', 'lanternfam', 'shazamfam']
['nah']
family_affil
Bat        9
Nope       9
Arrow      7
Flash      6
Super      4
Lantern    4
Wonder     2
Martian    2
Aqua       1
Beetle     1
Shazam     1
Name: count, dtype: int64


[]
['titans', 'jlu', 'outlaw', 'league_los', 'birds', 'yj', 'jld', 'jsa', 'lantern_corp', 'rogues', 'teen_justice']
['nah']
team_affil
Nope                                                    7
Teen Titans                                             6
Justice League                                          5
Teen Titans, Young Justice                              5
Young Justice                                           3
Justice Society                                         3
Green Lanterns, Justice League                          3
Rogue Gallery                                           3
Birds of Prey                                           2
Young Justice

In [1249]:
# Fix rarely selected options/blanks

for col, (type, other_check, dichot, split, changed, wrong) in threshold_dict.items():
    if "Multi" in type:
        multi_check = True
    else:
        multi_check = False
    if "write" in type:
        col = f'{col}_combined'
    print(f'{col} starting')
    print(f'--------------- M: {multi_check}')
    df = convert_to_other_threshold(df, col, multi_check)
    print('\n')

family_affil starting
--------------- M: True
{'Bat': 9, 'Nope': 9, 'Arrow': 7, 'Flash': 6, 'Super': 4, 'Lantern': 4, 'Wonder': 2, 'Martian': 2, 'Aqua': 1, 'Beetle': 1, 'Shazam': 1}
Aqua converted to other with 1 counts
Beetle converted to other with 1 counts
Shazam converted to other with 1 counts


base starting
--------------- M: False
base
Gotham             11
Keystone (City)     5
New York            4
Star (City)         4
Metropolis          2
Manchester          2
San Francisco       2
Bludhaven           1
Capitol (City)      1
Portland            1
Los Angeles         1
Philadelphia        1
Brooklyn            1
Virginia            1
Shadowcrest         1
Baltimore           1
Coast (City)        1
El Paso             1
Watchtower          1
Smallville          1
Atlantis            1
Happy Harbor        1
Name: count, dtype: int64
Bludhaven converted to other with 1 counts
Capitol (City) converted to other with 1 counts
Portland converted to other with 1 counts
Los Angeles

In [1250]:
# T/F ing

In [1251]:
for col, (other_check, dichotomy) in multi_write_sheets.items():

    filtered_sheets = [sheet for sheet in sheet_names if sheet.startswith(col)]
    base = zipit(filtered_sheets[0])
    if len(filtered_sheets) == 2:
        new_dict = zipit(filtered_sheets[1])
    else:
        new_dict = {}

    if other_check == "T":
        just_add_other = True
    else:
        just_add_other = False
    
    df = multiselect_tf(df, col, base, new_dict, just_add_other)
    print(f"{col} t/fed")

alignment_all_combined_other: ['Other'] done
alignment_all_combined_neutral: ['Neutral'] done
alignment_all_combined_evil: ["I'm evil"] done
alignment_all_combined_good: ["I'm good"] done
alignment_all t/fed
lethal_thoughts_combined_other: ['Other'] done
lethal_thoughts_combined_except: ['Some exceptions'] done
lethal_thoughts_combined_always: ['Always'] done
lethal_thoughts_combined_only_danger: ['Only if dangerous'] done
lethal_thoughts_combined_only_last: ['Only as last resort'] done
lethal_thoughts_combined_never: ['Never'] done
lethal_thoughts t/fed
race_combined_other: ['Other'] done
race_combined_magic: ['Magic species'] done
race_combined_alien_other: ['Other alien'] done
race_combined_kryptonian: ['Kryptonian'] done
race_combined_martian: ['Martian'] done
race_combined_atlantean: ['Atlantean'] done
race_combined_human: ['Human'] done
race t/fed


In [1252]:
for col in multi_basic_sheets:
    filtered_sheets = [sheet for sheet in sheet_names if sheet.startswith(col)]
    base = zipit(filtered_sheets[0])

    if col in changed_cols:
        base_2 = {v : k for k, v in base.items()}
        base = {v : k for k, v in base_2.items()}

    df = multi_select_no_other(df, col, base)
    print(f"{col} t/fed")

df.head()

family_affil_nah: ['Nope'] done
family_affil_shazamfam: ['Shazam'] done
family_affil_lanternfam: ['Lantern'] done
family_affil_beetlefam: ['Beetle'] done
family_affil_martianfam: ['Martian'] done
family_affil_superfam: ['Super'] done
family_affil_flashfam: ['Flash'] done
family_affil_arrowfam: ['Arrow'] done
family_affil_aquafam: ['Aqua'] done
family_affil_wonderfam: ['Wonder'] done
family_affil_batfam: ['Bat'] done
family_affil t/fed
team_affil_nah: ['Nope'] done
team_affil_teen_justice: ['Teen Justice'] done
team_affil_rogues: ['Rogue Gallery'] done
team_affil_lantern_corp: ['Green Lanterns'] done
team_affil_jsa: ['Justice Society'] done
team_affil_jld: ['Justice League Dark'] done
team_affil_yj: ['Young Justice'] done
team_affil_birds: ['Birds of Prey'] done
team_affil_league_los: ['League of Shadows'] done
team_affil_outlaw: ['Outlaws'] done
team_affil_jlu: ['Justice League'] done
team_affil_titans: ['Teen Titans'] done
team_affil t/fed
fight_why_nah: ['Not applicable', ' I no long

,ID,hero_or_villain,hero_or_villain_combined,alignment_all,alignment_all_combined,alignment_all_combined_good,alignment_all_combined_evil,alignment_all_combined_neutral,alignment_all_combined_other,family_affil,family_affil_batfam,family_affil_wonderfam,family_affil_aquafam,family_affil_arrowfam,family_affil_flashfam,family_affil_superfam,family_affil_martianfam,family_affil_beetlefam,family_affil_lanternfam,family_affil_shazamfam,family_affil_nah,team_affil,team_affil_titans,team_affil_jlu,team_affil_outlaw,team_affil_league_los,team_affil_birds,team_affil_yj,team_affil_jld,team_affil_jsa,team_affil_lantern_corp,team_affil_rogues,team_affil_teen_justice,team_affil_nah,sidekick_status,sidekick_status_combined,base,fight_why,fight_why_justice,fight_why_revenge,fight_why_responsibility,fight_why_redemption,fight_why_protection,fight_why_honor,fight_why_nan,fight_why_nah,year_start,main_weapon,main_weapon_combined,meta_powers,meta_powers_nah,meta_powers_tech,meta_powers_super,meta_powers_flight,meta_powers_race,meta_powers_speed,meta_powers_artifact,meta_powers_magic,main_rival,lethal_thoughts,lethal_thoughts_combined,lethal_thoughts_combined_never,lethal_thoughts_combined_only_last,lethal_thoughts_combined_only_danger,lethal_thoughts_combined_always,lethal_thoughts_combined_except,lethal_thoughts_combined_other,age,race,race_combined,race_combined_human,race_combined_atlantean,race_combined_martian,race_combined_kryptonian,race_combined_alien_other,race_combined_magic,race_combined_other,gender,gender_combined,orphan_status,trinity_1,trinity_2,trinity_3
0,1,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,0,0,0,"Teen Titans, Justice League",1,1,0,0,0,0,0,0,0,0,0,0,No,No,Other,I fight for justice,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1940,Escrima,Other,No powers,1,0,0,0,0,0,0,0,"Yes, very dangerous","never, only_last, but you sometimes think it'd be easier","Only as last resort, Other",0,1,0,0,0,1,29.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,0.0,M,M,Yes,Batman,Superman,Wonder Woman
1,2,You tell me,NaN,"good, evil, Complicated, Lazarus","I'm good, I'm evil, Other",1,1,0,1,Bat,1,0,0,0,0,0,0,0,0,0,0,"Outlaws, League of Shadows",0,0,1,1,0,0,0,0,0,0,0,0,No,No,Gotham,"I fight for justice, I fight for revenge",1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1983,Gun,Gun,No powers,1,0,0,0,0,0,0,0,"Yes, very dangerous","always, only_danger, some people just deserve it",Always,0,0,0,1,0,0,24.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,0.0,M,M,Yes,Wonder Woman,Superman,Wonder Woman
2,3,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,0,0,0,"Teen Titans, Young Justice",1,0,0,0,0,1,0,0,0,0,0,0,No? ?,No,Gotham,"I fight for justice, I fight for responsibility",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1989,Bo staff,Other,No powers,1,0,0,0,0,0,0,0,"Yes, very dangerous",never,Never,1,0,0,0,0,0,20.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,0.0,M,M,Yes,Batman,NaN,NaN
3,4,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,0,0,0,"Teen Titans, League of Shadows",1,0,0,1,0,0,0,0,0,0,0,0,tt Yes,Yes,Gotham,"I fight for justice, I fight for redemption",1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2006,Knife,Knife,No powers,1,0,0,0,0,0,0,0,"No, I don't","only_last, only for people who really do deserve it","Only as last resort, Some exceptions",0,1,0,0,1,0,13.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,0.0,M,M,No,Batman,Superman,NaN
4,5,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,0,0,0,Nope,0,0,0,0,0,0,0,0,0,0,0,1,No,No,Gotham,"I fight for justice, I fight for responsibility",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1992,Three-part staff,Other,No powers,1,0,0,0,0,0,0,0,"No, I don't","only_last, only_danger, but you can't take it back","Only as last resort, Only if dangerous, Other",0,1,1,0,0,1,21.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,0.0,F,F,No,Wonder Woman,NaN,NaN


In [1253]:
# T/F check/drop

In [1254]:
#Checks the t/f col counts

df_tf_count = df.apply(pd.to_numeric, errors='coerce')
for col in df_tf_count.columns:
    if df_tf_count[col].dtype == 'object':
        continue
    col_sum = df_tf_count[col].sum()
    if "combined_" in col and col_sum == 0:
        print(f'{col}: {col_sum}')
        df.pop(col)
    for sheet in multi_basic_sheets:
        if sheet in col and not col.endswith(sheet) and col_sum == 0:
            print(f'{col}: {col_sum}')
            df.pop(col)

family_affil_aquafam: 0
family_affil_beetlefam: 0
family_affil_shazamfam: 0
race_combined_atlantean: 0.0


In [1255]:
#Checking counts for "combined" cols (t/f)

df_2 = df.apply(pd.to_numeric, errors='coerce')
for col in df_2.columns:
    if "combined_" in col:
        col_sum = df_2[col].sum()
        print(f'{col}: {col_sum}')
    for sheet in multi_basic_sheets:
        if sheet in col and not col.endswith(sheet):
            col_sum = df_2[col].sum()
            print(f'{col}: {col_sum}')

alignment_all_combined_good: 41
alignment_all_combined_evil: 4
alignment_all_combined_neutral: 4
alignment_all_combined_other: 4
family_affil_batfam: 9
family_affil_wonderfam: 2
family_affil_arrowfam: 7
family_affil_flashfam: 6
family_affil_superfam: 4
family_affil_martianfam: 2
family_affil_lanternfam: 4
family_affil_nah: 9
team_affil_titans: 14
team_affil_jlu: 11
team_affil_outlaw: 2
team_affil_league_los: 3
team_affil_birds: 2
team_affil_yj: 9
team_affil_jld: 1
team_affil_jsa: 5
team_affil_lantern_corp: 4
team_affil_rogues: 3
team_affil_teen_justice: 1
team_affil_nah: 7
fight_why_justice: 25.0
fight_why_revenge: 5.0
fight_why_responsibility: 17.0
fight_why_redemption: 5.0
fight_why_protection: 5.0
fight_why_honor: 11.0
fight_why_nan: 2.0
fight_why_nah: 4.0
meta_powers_nah: 17
meta_powers_tech: 5
meta_powers_super: 8
meta_powers_flight: 15
meta_powers_race: 11
meta_powers_speed: 6
meta_powers_artifact: 2
meta_powers_magic: 3
lethal_thoughts_combined_never: 12
lethal_thoughts_combined

In [1256]:
#Graphs for the multi-select and multi-write sheets: visually check #s
#Leave off unless checking

'''multi_baby_sheets = {**multi_basic_sheets, **multi_write_sheets}

for q in multi_baby_sheets:
    df_copy = df.copy()
    filtered_list = [col for col in df_copy.columns if col.startswith(f'{q}_') and not col.endswith("combined") and not col.endswith("yn_split")]

    print(filtered_list)

    df_multibabychart = df_copy[filtered_list]
    df_multibabychart = df_multibabychart.apply(pd.to_numeric, errors='coerce')

    yes_counts = df_multibabychart.sum()
    total_responses = len(df_multibabychart)
    percentages = yes_counts / total_responses * 100

    # Plotting
    plt.figure(figsize=(8, 3))
    ax = yes_counts.plot(kind='bar', color='skyblue')

    new_labels = [col.replace(f'{q}_combined_', '') for col in filtered_list]
    new_labels = [col.replace(f'{q}_', '') for col in new_labels]
    ax.set_xticklabels(new_labels, rotation=45, ha='right')

    for i, v in enumerate(percentages):
        ax.text(i, v + 1, f"{percentages.iloc[i]:.1f}%", ha='center', va='bottom', fontsize=9)

    plt.title(f'{q}')
    plt.ylabel('Count')
    plt.xticks(rotation=45)

    plt.tight_layout()
    plt.show()'''

'multi_baby_sheets = {**multi_basic_sheets, **multi_write_sheets}\n\nfor q in multi_baby_sheets:\n    df_copy = df.copy()\n    filtered_list = [col for col in df_copy.columns if col.startswith(f\'{q}_\') and not col.endswith("combined") and not col.endswith("yn_split")]\n\n    print(filtered_list)\n\n    df_multibabychart = df_copy[filtered_list]\n    df_multibabychart = df_multibabychart.apply(pd.to_numeric, errors=\'coerce\')\n\n    yes_counts = df_multibabychart.sum()\n    total_responses = len(df_multibabychart)\n    percentages = yes_counts / total_responses * 100\n\n    # Plotting\n    plt.figure(figsize=(8, 3))\n    ax = yes_counts.plot(kind=\'bar\', color=\'skyblue\')\n\n    new_labels = [col.replace(f\'{q}_combined_\', \'\') for col in filtered_list]\n    new_labels = [col.replace(f\'{q}_\', \'\') for col in new_labels]\n    ax.set_xticklabels(new_labels, rotation=45, ha=\'right\')\n\n    for i, v in enumerate(percentages):\n        ax.text(i, v + 1, f"{percentages.iloc[i]:.1f

In [1257]:
#Graphs for the single-select and bool sheets: visually check #s
#Leave off unless checking

'''single_baby_sheets = {**single_basic_sheets, **bool_sheets}

for col in single_baby_sheets:    
    value_counts = df[f'{col}'].value_counts()
    print(value_counts.keys())
    
    total_responses = df[f'{col}'].count()
    percentages = value_counts / total_responses * 100

    #PLotting
    plt.figure(figsize=(6, 5))

    ax = value_counts.plot(kind='bar', color='skyblue')
    
    for i, v in enumerate(value_counts):
        ax.text(i, v + 1, f"{percentages.iloc[i]:.1f}%", ha='center', va='bottom', fontsize=9)
    
    plt.title(f'{col}')
    plt.ylabel('Count')
    plt.xlabel('')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()'''

'single_baby_sheets = {**single_basic_sheets, **bool_sheets}\n\nfor col in single_baby_sheets:    \n    value_counts = df[f\'{col}\'].value_counts()\n    print(value_counts.keys())\n    \n    total_responses = df[f\'{col}\'].count()\n    percentages = value_counts / total_responses * 100\n\n    #PLotting\n    plt.figure(figsize=(6, 5))\n\n    ax = value_counts.plot(kind=\'bar\', color=\'skyblue\')\n    \n    for i, v in enumerate(value_counts):\n        ax.text(i, v + 1, f"{percentages.iloc[i]:.1f}%", ha=\'center\', va=\'bottom\', fontsize=9)\n    \n    plt.title(f\'{col}\')\n    plt.ylabel(\'Count\')\n    plt.xlabel(\'\')\n    plt.xticks(rotation=45)\n    \n    plt.tight_layout()\n    plt.show()'

In [1258]:
#Graphs for the single-write sheets: visually check #s
#Leave off unless checking

'''for col in single_write_sheets:
    
    value_counts = df[f'{col}_combined'].value_counts()
    print(value_counts.keys())
    
    total_responses = df[f'{col}_combined'].count()
    percentages = value_counts / total_responses * 100

    #Plotting

    plt.figure(figsize=(6, 5))
    
    ax = value_counts.plot(kind='bar', color='skyblue')
    
    for i, v in enumerate(value_counts):
        ax.text(i, v + 1, f"{percentages.iloc[i]:.1f}%", ha='center', va='bottom', fontsize=9)
    
    plt.title(f'{col}')
    plt.ylabel('Count')
    plt.xlabel('')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()'''

'for col in single_write_sheets:\n    \n    value_counts = df[f\'{col}_combined\'].value_counts()\n    print(value_counts.keys())\n    \n    total_responses = df[f\'{col}_combined\'].count()\n    percentages = value_counts / total_responses * 100\n\n    #Plotting\n\n    plt.figure(figsize=(6, 5))\n    \n    ax = value_counts.plot(kind=\'bar\', color=\'skyblue\')\n    \n    for i, v in enumerate(value_counts):\n        ax.text(i, v + 1, f"{percentages.iloc[i]:.1f}%", ha=\'center\', va=\'bottom\', fontsize=9)\n    \n    plt.title(f\'{col}\')\n    plt.ylabel(\'Count\')\n    plt.xlabel(\'\')\n    plt.xticks(rotation=45)\n    \n    plt.tight_layout()\n    plt.show()'

In [1259]:
df.head()

,ID,hero_or_villain,hero_or_villain_combined,alignment_all,alignment_all_combined,alignment_all_combined_good,alignment_all_combined_evil,alignment_all_combined_neutral,alignment_all_combined_other,family_affil,family_affil_batfam,family_affil_wonderfam,family_affil_arrowfam,family_affil_flashfam,family_affil_superfam,family_affil_martianfam,family_affil_lanternfam,family_affil_nah,team_affil,team_affil_titans,team_affil_jlu,team_affil_outlaw,team_affil_league_los,team_affil_birds,team_affil_yj,team_affil_jld,team_affil_jsa,team_affil_lantern_corp,team_affil_rogues,team_affil_teen_justice,team_affil_nah,sidekick_status,sidekick_status_combined,base,fight_why,fight_why_justice,fight_why_revenge,fight_why_responsibility,fight_why_redemption,fight_why_protection,fight_why_honor,fight_why_nan,fight_why_nah,year_start,main_weapon,main_weapon_combined,meta_powers,meta_powers_nah,meta_powers_tech,meta_powers_super,meta_powers_flight,meta_powers_race,meta_powers_speed,meta_powers_artifact,meta_powers_magic,main_rival,lethal_thoughts,lethal_thoughts_combined,lethal_thoughts_combined_never,lethal_thoughts_combined_only_last,lethal_thoughts_combined_only_danger,lethal_thoughts_combined_always,lethal_thoughts_combined_except,lethal_thoughts_combined_other,age,race,race_combined,race_combined_human,race_combined_martian,race_combined_kryptonian,race_combined_alien_other,race_combined_magic,race_combined_other,gender,gender_combined,orphan_status,trinity_1,trinity_2,trinity_3
0,1,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,"Teen Titans, Justice League",1,1,0,0,0,0,0,0,0,0,0,0,No,No,Other,I fight for justice,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1940,Escrima,Other,No powers,1,0,0,0,0,0,0,0,"Yes, very dangerous","never, only_last, but you sometimes think it'd be easier","Only as last resort, Other",0,1,0,0,0,1,29.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,M,M,Yes,Batman,Superman,Wonder Woman
1,2,You tell me,NaN,"good, evil, Complicated, Lazarus","I'm good, I'm evil, Other",1,1,0,1,Bat,1,0,0,0,0,0,0,0,"Outlaws, League of Shadows",0,0,1,1,0,0,0,0,0,0,0,0,No,No,Gotham,"I fight for justice, I fight for revenge",1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1983,Gun,Gun,No powers,1,0,0,0,0,0,0,0,"Yes, very dangerous","always, only_danger, some people just deserve it",Always,0,0,0,1,0,0,24.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,M,M,Yes,Wonder Woman,Superman,Wonder Woman
2,3,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,"Teen Titans, Young Justice",1,0,0,0,0,1,0,0,0,0,0,0,No? ?,No,Gotham,"I fight for justice, I fight for responsibility",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1989,Bo staff,Other,No powers,1,0,0,0,0,0,0,0,"Yes, very dangerous",never,Never,1,0,0,0,0,0,20.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,M,M,Yes,Batman,NaN,NaN
3,4,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,"Teen Titans, League of Shadows",1,0,0,1,0,0,0,0,0,0,0,0,tt Yes,Yes,Gotham,"I fight for justice, I fight for redemption",1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2006,Knife,Knife,No powers,1,0,0,0,0,0,0,0,"No, I don't","only_last, only for people who really do deserve it","Only as last resort, Some exceptions",0,1,0,0,1,0,13.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,M,M,No,Batman,Superman,NaN
4,5,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,Nope,0,0,0,0,0,0,0,0,0,0,0,1,No,No,Gotham,"I fight for justice, I fight for responsibility",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1992,Three-part staff,Other,No powers,1,0,0,0,0,0,0,0,"No, I don't","only_last, only_danger, but you can't take it back","Only as last resort, Only if dangerous, Other",0,1,1,0,0,1,21.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,F,F,No,Wonder Woman,NaN,NaN


In [1260]:
# Mutations

In [1261]:
#Rankings
#Because initially it was choose element for ranking, not ranking for element, this needs to be reversed

trinity_scores = [col for col in df.columns if col.startswith("trinity")]

combined_series = pd.concat([df[col] for col in trinity_scores])
combined_series = combined_series.dropna()
trinity_members = combined_series.unique()
trinity_members = trinity_members.tolist()

results = {}

for member in trinity_members:
    result_list = []
    
    for index in df.index:
        found_in_column = np.nan
        for i in range(len(trinity_scores)):
            if df.loc[index, trinity_scores[i]] == member:
                found_in_column = 3 - i
                break
        result_list.append(found_in_column)
    
    results[f'{member}_score'] = result_list
new_columns_df = pd.DataFrame(results, index=df.index)
df = pd.concat([df, new_columns_df], axis=1)

In [1262]:
df.head()

,ID,hero_or_villain,hero_or_villain_combined,alignment_all,alignment_all_combined,alignment_all_combined_good,alignment_all_combined_evil,alignment_all_combined_neutral,alignment_all_combined_other,family_affil,family_affil_batfam,family_affil_wonderfam,family_affil_arrowfam,family_affil_flashfam,family_affil_superfam,family_affil_martianfam,family_affil_lanternfam,family_affil_nah,team_affil,team_affil_titans,team_affil_jlu,team_affil_outlaw,team_affil_league_los,team_affil_birds,team_affil_yj,team_affil_jld,team_affil_jsa,team_affil_lantern_corp,team_affil_rogues,team_affil_teen_justice,team_affil_nah,sidekick_status,sidekick_status_combined,base,fight_why,fight_why_justice,fight_why_revenge,fight_why_responsibility,fight_why_redemption,fight_why_protection,fight_why_honor,fight_why_nan,fight_why_nah,year_start,main_weapon,main_weapon_combined,meta_powers,meta_powers_nah,meta_powers_tech,meta_powers_super,meta_powers_flight,meta_powers_race,meta_powers_speed,meta_powers_artifact,meta_powers_magic,main_rival,lethal_thoughts,lethal_thoughts_combined,lethal_thoughts_combined_never,lethal_thoughts_combined_only_last,lethal_thoughts_combined_only_danger,lethal_thoughts_combined_always,lethal_thoughts_combined_except,lethal_thoughts_combined_other,age,race,race_combined,race_combined_human,race_combined_martian,race_combined_kryptonian,race_combined_alien_other,race_combined_magic,race_combined_other,gender,gender_combined,orphan_status,trinity_1,trinity_2,trinity_3,Batman_score,Wonder Woman_score,Superman_score
0,1,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,"Teen Titans, Justice League",1,1,0,0,0,0,0,0,0,0,0,0,No,No,Other,I fight for justice,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1940,Escrima,Other,No powers,1,0,0,0,0,0,0,0,"Yes, very dangerous","never, only_last, but you sometimes think it'd be easier","Only as last resort, Other",0,1,0,0,0,1,29.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,M,M,Yes,Batman,Superman,Wonder Woman,3.0,1.0,2.0
1,2,You tell me,NaN,"good, evil, Complicated, Lazarus","I'm good, I'm evil, Other",1,1,0,1,Bat,1,0,0,0,0,0,0,0,"Outlaws, League of Shadows",0,0,1,1,0,0,0,0,0,0,0,0,No,No,Gotham,"I fight for justice, I fight for revenge",1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1983,Gun,Gun,No powers,1,0,0,0,0,0,0,0,"Yes, very dangerous","always, only_danger, some people just deserve it",Always,0,0,0,1,0,0,24.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,M,M,Yes,Wonder Woman,Superman,Wonder Woman,NaN,3.0,2.0
2,3,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,"Teen Titans, Young Justice",1,0,0,0,0,1,0,0,0,0,0,0,No? ?,No,Gotham,"I fight for justice, I fight for responsibility",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1989,Bo staff,Other,No powers,1,0,0,0,0,0,0,0,"Yes, very dangerous",never,Never,1,0,0,0,0,0,20.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,M,M,Yes,Batman,NaN,NaN,3.0,NaN,NaN
3,4,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,"Teen Titans, League of Shadows",1,0,0,1,0,0,0,0,0,0,0,0,tt Yes,Yes,Gotham,"I fight for justice, I fight for redemption",1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2006,Knife,Knife,No powers,1,0,0,0,0,0,0,0,"No, I don't","only_last, only for people who really do deserve it","Only as last resort, Some exceptions",0,1,0,0,1,0,13.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,M,M,No,Batman,Superman,NaN,3.0,NaN,2.0
4,5,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,Nope,0,0,0,0,0,0,0,0,0,0,0,1,No,No,Gotham,"I fight for justice, I fight for responsibility",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1992,Three-part staff,Other,No powers,1,0,0,0,0,0,0,0,"No, I don't","only_last, only_danger, but you can't take it back","Only as last resort, Only if dangerous, Other",0,1,1,0,0,1,21.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,F,F,No,Wonder Woman,NaN,NaN,NaN,3.0,NaN


In [1263]:
# Split off cols, for cols that are secretly two in one

#Risks of "other" appearing from threshold lol

for col in split_me_cols:
    sheet_dict =  pd.read_excel(big_data_dict, col, header=None)
    replace_dict = dict(zip(sheet_dict[0],
                                sheet_dict[2]))

    df[f'{col}_yn_split']=df[col].replace(replace_dict, regex=False)
    df = col_index_move(df, col, f'{col}_yn_split')

    df[f'{col}_yn_split'] = df[f'{col}_yn_split'].apply(replace_bool)

    print(f"{col} splitted")
    print(df[f'{col}'].value_counts())
    print("")
    print(df[f'{col}_yn_split'].value_counts())
    print("\n")


main_rival splitted
main_rival
No, I don't            29
Yes, very dangerous    11
Yes, very annoying      6
Name: count, dtype: int64

main_rival_yn_split
No     29
Yes    17
Name: count, dtype: int64




In [1264]:
df.head()

,ID,hero_or_villain,hero_or_villain_combined,alignment_all,alignment_all_combined,alignment_all_combined_good,alignment_all_combined_evil,alignment_all_combined_neutral,alignment_all_combined_other,family_affil,family_affil_batfam,family_affil_wonderfam,family_affil_arrowfam,family_affil_flashfam,family_affil_superfam,family_affil_martianfam,family_affil_lanternfam,family_affil_nah,team_affil,team_affil_titans,team_affil_jlu,team_affil_outlaw,team_affil_league_los,team_affil_birds,team_affil_yj,team_affil_jld,team_affil_jsa,team_affil_lantern_corp,team_affil_rogues,team_affil_teen_justice,team_affil_nah,sidekick_status,sidekick_status_combined,base,fight_why,fight_why_justice,fight_why_revenge,fight_why_responsibility,fight_why_redemption,fight_why_protection,fight_why_honor,fight_why_nan,fight_why_nah,year_start,main_weapon,main_weapon_combined,meta_powers,meta_powers_nah,meta_powers_tech,meta_powers_super,meta_powers_flight,meta_powers_race,meta_powers_speed,meta_powers_artifact,meta_powers_magic,main_rival,main_rival_yn_split,lethal_thoughts,lethal_thoughts_combined,lethal_thoughts_combined_never,lethal_thoughts_combined_only_last,lethal_thoughts_combined_only_danger,lethal_thoughts_combined_always,lethal_thoughts_combined_except,lethal_thoughts_combined_other,age,race,race_combined,race_combined_human,race_combined_martian,race_combined_kryptonian,race_combined_alien_other,race_combined_magic,race_combined_other,gender,gender_combined,orphan_status,trinity_1,trinity_2,trinity_3,Batman_score,Wonder Woman_score,Superman_score
0,1,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,"Teen Titans, Justice League",1,1,0,0,0,0,0,0,0,0,0,0,No,No,Other,I fight for justice,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1940,Escrima,Other,No powers,1,0,0,0,0,0,0,0,"Yes, very dangerous",Yes,"never, only_last, but you sometimes think it'd be easier","Only as last resort, Other",0,1,0,0,0,1,29.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,M,M,Yes,Batman,Superman,Wonder Woman,3.0,1.0,2.0
1,2,You tell me,NaN,"good, evil, Complicated, Lazarus","I'm good, I'm evil, Other",1,1,0,1,Bat,1,0,0,0,0,0,0,0,"Outlaws, League of Shadows",0,0,1,1,0,0,0,0,0,0,0,0,No,No,Gotham,"I fight for justice, I fight for revenge",1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1983,Gun,Gun,No powers,1,0,0,0,0,0,0,0,"Yes, very dangerous",Yes,"always, only_danger, some people just deserve it",Always,0,0,0,1,0,0,24.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,M,M,Yes,Wonder Woman,Superman,Wonder Woman,NaN,3.0,2.0
2,3,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,"Teen Titans, Young Justice",1,0,0,0,0,1,0,0,0,0,0,0,No? ?,No,Gotham,"I fight for justice, I fight for responsibility",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1989,Bo staff,Other,No powers,1,0,0,0,0,0,0,0,"Yes, very dangerous",Yes,never,Never,1,0,0,0,0,0,20.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,M,M,Yes,Batman,NaN,NaN,3.0,NaN,NaN
3,4,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,"Teen Titans, League of Shadows",1,0,0,1,0,0,0,0,0,0,0,0,tt Yes,Yes,Gotham,"I fight for justice, I fight for redemption",1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2006,Knife,Knife,No powers,1,0,0,0,0,0,0,0,"No, I don't",No,"only_last, only for people who really do deserve it","Only as last resort, Some exceptions",0,1,0,0,1,0,13.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,M,M,No,Batman,Superman,NaN,3.0,NaN,2.0
4,5,Hero,Hero,good,I'm good,1,0,0,0,Bat,1,0,0,0,0,0,0,0,Nope,0,0,0,0,0,0,0,0,0,0,0,1,No,No,Gotham,"I fight for justice, I fight for responsibility",1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1992,Three-part staff,Other,No powers,1,0,0,0,0,0,0,0,"No, I don't",No,"only_last, only_danger, but you can't take it back","Only as last resort, Only if dangerous, Other",0,1,1,0,0,1,21.0,human,Human,1.0,0.0,0.0,0.0,0.0,0.0,F,F,No,Wonder Woman,NaN,NaN,NaN,3.0,NaN


In [1265]:
#Check the write ins and their kids

filtered_combined_cols = filter_columns(df, "combined")

for col in filtered_combined_cols:
    print(col)
    for value, count in df[col].value_counts().items():
        if count < 10:
            print(f'{count} counts: {value}')

hero_or_villain_combined
3 counts: Villain
2 counts: Civilian
2 counts: Other
alignment_all_combined
2 counts: I'm good, Other
2 counts: I'm evil
1 counts: I'm good, I'm evil, Other
1 counts: Other
1 counts: Neutral, I'm evil
1 counts: Neutral, I'm good
1 counts: Neutral
1 counts: I'm good, Neutral
alignment_all_combined_good
5 counts: 0
alignment_all_combined_evil
4 counts: 1
alignment_all_combined_neutral
4 counts: 1
alignment_all_combined_other
4 counts: 1
sidekick_status_combined
6 counts: Yes
main_weapon_combined
8 counts: Other
8 counts: Meta superpowers
7 counts: Tech
5 counts: Hand to hand
5 counts: Arrows
5 counts: Speedforce
2 counts: Gun
2 counts: Knife
2 counts: Magic
2 counts: Not applicable, I no longer/do not fight
lethal_thoughts_combined
7 counts: Always
7 counts: Only if dangerous
5 counts: Only as last resort, Only if dangerous
1 counts: Only as last resort, Other
1 counts: Only as last resort, Some exceptions
1 counts: Only as last resort, Only if dangerous, Other
1

In [1266]:
#Check wide view

for col in df.columns:
    if col == "ID":
        continue
    print(f"Unique values for column '{col}':")
    print(df[col].unique())
    print()

Unique values for column 'hero_or_villain':
['Hero' 'You tell me' 'Villain' 'Civilian' 'vigilante' 'dating one?'
 'retired now!']

Unique values for column 'hero_or_villain_combined':
['Hero' nan 'Villain' 'Civilian' 'Other']

Unique values for column 'alignment_all':
['good' 'good, evil, Complicated, Lazarus'
 'good, mind controlled? I think, that counts' 'I try my best'
 'good, Chaotic' 'neutral, evil' 'neutral, good' 'evil' 'neutral'
 'good, neutral']

Unique values for column 'alignment_all_combined':
["I'm good" "I'm good, I'm evil, Other" "I'm good, Other" 'Other'
 "Neutral, I'm evil" "Neutral, I'm good" "I'm evil" 'Neutral'
 "I'm good, Neutral"]

Unique values for column 'alignment_all_combined_good':
[1 0]

Unique values for column 'alignment_all_combined_evil':
[0 1]

Unique values for column 'alignment_all_combined_neutral':
[0 1]

Unique values for column 'alignment_all_combined_other':
[0 1]

Unique values for column 'family_affil':
['Bat' 'Wonder' 'Other' 'Arrow' 'Flash' '

In [1267]:
df.to_csv("cleanedfile.csv")